## Applied Data Science Capstone - Week 3 - Third Link

### Website scraping with BeautifulSoup

In [639]:

# Beautiful Soup
from bs4 import BeautifulSoup
import requests

# url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Getting the webpage, creating a Response object.
response = requests.get(url)
 
# Extracting the source code of the page.
data = response.text
 
# Passing the source code to BeautifulSoup to create a BeautifulSoup object for it.
soup = BeautifulSoup(data, 'lxml')

# Find the Table
My_table = soup.find('table',{'class':'wikitable sortable'})

### Create the DataFrame

In [640]:
# import Pandas
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood']

# extract information from My_table
rows_list = []
for tr in My_table.findAll('tr'):
    tds = tr.findAll('td')
    if not tds:
        continue
    postcode, borough, neighborhood = [td.text.strip() for td in tds[:3]]
    rows_list.append([postcode, borough, neighborhood])

# create dataframe (without cleaning)
postalcodes = pd.DataFrame(rows_list, columns=column_names)

### Clean the DataFrame and show the first 5 rows of the resulting DataFrame

In [641]:
# Drop rows where 'Borough' == 'Not assigned'
indexNames = postalcodes[postalcodes['Borough'] == 'Not assigned'].index
postalcodes.drop(indexNames, inplace=True)
postalcodes = postalcodes.reset_index()
del postalcodes['index']

# Assign 'Borough' where 'Neighborhood' == Not assigned'
postalcodes.loc[postalcodes['Neighborhood'] == ('Not assigned'), 'Neighborhood'] = postalcodes['Borough']

# Combine Neighborhoods in rows with the same PostalCode and Borough
postalcodes_group = postalcodes.groupby(['PostalCode', 'Borough'], sort=False).agg(lambda x: ', '.join(x))
postalcodes_group = postalcodes_group.reset_index()

# Show the first 5 rows from the new DataFrame
postalcodes_group.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


### Print the number of rows of the resulting DataFrame.

In [642]:
number_of_rows = postalcodes_group.shape[0]
print('The number of rows is ', number_of_rows)

The number of rows is  103


## Get the latitude and the longitude coordinates of each neighborhood, from .csv file

### Read the .csv file

In [643]:
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Rename 'Postal Code'column to be 'PostalCode'as in the previous DataFrame

In [644]:
coordinates_df.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
coordinates_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge both DataFrames, based on the first one

In [645]:
merged_dataframe = pd.merge(postalcodes_group, coordinates_df, how='left', on='PostalCode')
merged_dataframe.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# Explore and cluster the neighborhoods in Toronto.

### Import Libraries

In [646]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Chose to work with only boroughs that contain the word Toronto

In [647]:
merged_dataframe_Toronto = merged_dataframe[merged_dataframe['Borough'].str.contains("Toronto")]
merged_dataframe_Toronto = merged_dataframe_Toronto.reset_index()
del merged_dataframe_Toronto['index']
merged_dataframe_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [648]:
merged_dataframe_Toronto.shape

(38, 5)

In [649]:
dfToronto = merged_dataframe_Toronto
dfToronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Replicate the same analysis we did to the New York City data

## 1. Explore the Dataset

#### Use geopy library to get the latitude and longitude values of New York City.

In [650]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [651]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postalcode, borough, neighborhood in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['PostalCode'], dfToronto['Borough'], dfToronto['Neighborhood']):
    label = '{}, {}, {}'.format(postalcode, borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [652]:
CLIENT_ID = 'OJD3IGLCOWUZVX5KX32H44MUTGNVGL1AYL1D1GNHO0CTNGTS' # your Foursquare ID
CLIENT_SECRET = 'PNVBN13TA11AUDH1DKK4AVLUCJK3QJB0QKOVDNYSATBGNIFO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OJD3IGLCOWUZVX5KX32H44MUTGNVGL1AYL1D1GNHO0CTNGTS
CLIENT_SECRET:PNVBN13TA11AUDH1DKK4AVLUCJK3QJB0QKOVDNYSATBGNIFO


#### Let's explore the first Neighborhood in our dataframe.

Get the neighborhood's name.

In [653]:
dfToronto.loc[0, 'Neighborhood']

'Harbourfront, Regent Park'

Get the neighborhood's latitude and longitude values.

In [654]:
neighborhood_latitude = dfToronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfToronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfToronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront, Regent Park are 43.6542599, -79.3606359.


#### Now, let's get the top 100 venues that are in Harbourfront, Regent Park within a radius of 800 meters.

First, let's create the GET request URL. Name your URL **url**.

In [655]:
LIMIT = 100 # max venues to retrieve
radius = 800 # radius from Harbourfront, Regent Park's coordinates

# create URL and name ir "url" using the method "explore"
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OJD3IGLCOWUZVX5KX32H44MUTGNVGL1AYL1D1GNHO0CTNGTS&client_secret=PNVBN13TA11AUDH1DKK4AVLUCJK3QJB0QKOVDNYSATBGNIFO&v=20180605&ll=43.6542599,-79.3606359&radius=800&limit=100'

Send the GET request and examine the resutls

In [656]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca910604c1f67164c19e775'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 86,
  'suggestedBounds': {'ne': {'lat': 43.661459907200005,
    'lng': -79.35070310023775},
   'sw': {'lat': 43.647059892799994, 'lng': -79.37056869976226}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [657]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [658]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


And how many venues were returned by Foursquare?

In [659]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

86 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [660]:
def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])


    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [661]:
toronto_venues = getNearbyVenues(names=dfToronto['Neighborhood'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

#### Let's check the size of the resulting dataframe

In [662]:
print(toronto_venues.shape)
toronto_venues.head()

(2715, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's check how many venues were returned for each neighborhood

In [663]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",93,93,93,93,93,93
Business Reply Mail Processing Centre 969 Eastern,55,55,55,55,55,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",26,26,26,26,26,26
"Cabbagetown, St. James Town",75,75,75,75,75,75
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,63,63,63,63,63,63


#### Let's find out how many unique categories can be curated from all the returned venues

In [664]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 281 uniques categories.


## 3. Analyze Each Neighborhood

#### OBS: as there are some few venues (4) which have "Neighborhood" as their Venue_Category, I'll drop this venues from the dataframe !!!

In [665]:
# Drop rows where 'Venue Category' == 'Neighborhood'
indexNamesVC = toronto_venues[toronto_venues['Venue Category'] == 'Neighborhood'].index
toronto_venues.drop(indexNamesVC, inplace=True)
toronto_venues = toronto_venues.reset_index()
del toronto_venues['index']

In [666]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,W

And let's examine the new dataframe size.

In [667]:
toronto_onehot.shape

(2707, 281)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [668]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,W

#### Let's confirm the new size

In [669]:
toronto_grouped.shape

(38, 281)

#### Let's print each neighborhood along with the top 5 most common venues

In [670]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2   Steakhouse  0.04
3        Hotel  0.04
4    Gastropub  0.03


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.06
2                 Café  0.06
3           Restaurant  0.04
4  Japanese Restaurant  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.08
1                    Café  0.04
2                     Bar  0.04
3              Restaurant  0.04
4  Furniture / Home Store  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0  Fast Food Restaurant  0.07
1    Light Rail Station  0.07
2                   Bar  0.05
3               Brewery  0.04
4                Bakery  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0      Harbo

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [671]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    # print('Indice')
    # print(row_categories_sorted[0:num_top_venues])
    # print('Valor')
    # print(row_categories_sorted.index[0:num_top_venues])
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [672]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Bar,Theater,Gastropub,Sushi Restaurant,Restaurant,American Restaurant
1,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Bakery,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Park,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Furniture / Home Store,Restaurant,Café,Bar,Bakery,Arts & Crafts Store,Thrift / Vintage Store,Plaza,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Fast Food Restaurant,Bar,Pizza Place,Italian Restaurant,Brewery,Harbor / Marina,Bakery,Grocery Store,Burrito Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Harbor / Marina,Airport Terminal,Boat or Ferry,Airport Lounge,Sculpture Garden,Airport Service,Rental Car Location,Airport Gate,Coffee Shop,Music Venue


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [673]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 1, 4, 4, 4, 1, 1, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [674]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfToronto

# merge toronto_grouped with dfToronto (Toronto data) to add latitude/longitude for each neighborhood 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Café,Italian Restaurant,Theater,Restaurant,Pub,Thai Restaurant,Mexican Restaurant
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,4,Coffee Shop,Clothing Store,Café,Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Gastropub,Pizza Place,Japanese Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Bakery,Gastropub,Breakfast Spot,Cosmetics Shop,Seafood Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Coffee Shop,Bar,Gastropub,Japanese Restaurant,Breakfast Spot,Sandwich Place,Church,Electronics Store,Indian Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Coffee Shop,Hotel,Café,Restaurant,Bakery,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Park,Breakfast Spot


Finally, let's visualize the resulting clusters

In [675]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [678]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,0,Grocery Store,Park,Sushi Restaurant,Gym,Candy Store,Sandwich Place,Bank,Café,Japanese Restaurant,Tennis Court
32,Downtown Toronto,0,Park,Bank,Trail,Playground,Candy Store,Grocery Store,Comedy Club,Design Studio,Ethiopian Restaurant,Electronics Store


### Cluster 2

In [680]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,1,Pub,Coffee Shop,Bar,Gastropub,Japanese Restaurant,Breakfast Spot,Sandwich Place,Church,Electronics Store,Indian Restaurant
6,Downtown Toronto,1,Korean Restaurant,Grocery Store,Coffee Shop,Park,Diner,Café,Pizza Place,Japanese Restaurant,Bar,Indian Restaurant
8,West Toronto,1,Coffee Shop,Park,Gym,Bar,Pharmacy,Supermarket,Portuguese Restaurant,Discount Store,Bakery,Café
10,West Toronto,1,Bar,Café,Bakery,Coffee Shop,Cocktail Bar,Restaurant,Asian Restaurant,Italian Restaurant,Tapas Restaurant,Theater
14,East Toronto,1,Indian Restaurant,Sandwich Place,Gym,Café,Brewery,Italian Restaurant,Coffee Shop,Fast Food Restaurant,Snack Place,Diner
16,East Toronto,1,Café,Bakery,Bar,Coffee Shop,Sandwich Place,Gastropub,Italian Restaurant,Sushi Restaurant,Diner,Brewery
21,West Toronto,1,Bar,Café,Bakery,Italian Restaurant,Coffee Shop,Park,Grocery Store,Thai Restaurant,Mexican Restaurant,Antique Shop
24,West Toronto,1,Bar,Coffee Shop,Breakfast Spot,Pizza Place,Sushi Restaurant,Thai Restaurant,Dog Run,Restaurant,Eastern European Restaurant,Sandwich Place
26,Downtown Toronto,1,Café,Coffee Shop,Restaurant,Bakery,Vegetarian / Vegan Restaurant,Pub,Bookstore,Bubble Tea Shop,Burrito Place,Pizza Place
29,Downtown Toronto,1,Café,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Vietnamese Restaurant,Bakery,Chinese Restaurant,Dessert Shop,Mexican Restaurant,Ramen Restaurant


### Cluster 3

In [681]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Playground,Garden,Pool,Dog Run,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar


### Cluster 4

In [682]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,3,Coffee Shop,Park,Restaurant,Bus Line,Café,Gym / Fitness Center,Bookstore,Electronics Store,Eastern European Restaurant,Ethiopian Restaurant


### Cluster 5

In [683]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Park,Bakery,Café,Italian Restaurant,Theater,Restaurant,Pub,Thai Restaurant,Mexican Restaurant
1,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Cosmetics Shop,Gastropub,Pizza Place,Japanese Restaurant
2,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Hotel,Italian Restaurant,Bakery,Gastropub,Breakfast Spot,Cosmetics Shop,Seafood Restaurant
4,Downtown Toronto,4,Coffee Shop,Hotel,Café,Restaurant,Bakery,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Park,Breakfast Spot
5,Downtown Toronto,4,Coffee Shop,Bubble Tea Shop,Café,Art Gallery,Italian Restaurant,Japanese Restaurant,Bar,Ramen Restaurant,Arts & Crafts Store,Tea Room
7,Downtown Toronto,4,Coffee Shop,Café,Hotel,Steakhouse,Bar,Theater,Gastropub,Sushi Restaurant,Restaurant,American Restaurant
9,Downtown Toronto,4,Coffee Shop,Hotel,Italian Restaurant,Café,Aquarium,Restaurant,Park,Brewery,Japanese Restaurant,Scenic Lookout
11,East Toronto,4,Greek Restaurant,Coffee Shop,Pub,Fast Food Restaurant,Italian Restaurant,Grocery Store,Ice Cream Shop,Café,Yoga Studio,Pizza Place
12,Downtown Toronto,4,Café,Hotel,Coffee Shop,Restaurant,American Restaurant,Steakhouse,Gastropub,Deli / Bodega,Seafood Restaurant,Pizza Place
13,West Toronto,4,Coffee Shop,Furniture / Home Store,Restaurant,Café,Bar,Bakery,Arts & Crafts Store,Thrift / Vintage Store,Plaza,Caribbean Restaurant
